# ORB-SLAM3 Camera and IMU calibration Tutorial

In [1]:
import cv2
import numpy as np
import tempfile
import shutil
from tqdm.notebook import tqdm
from pathlib import Path

## Synchronize stereo camera frames

In [10]:
SYNC_TIME_DIFF_MAX = 100 # msec

# Read original dataset
ds_dir = Path("datasets/calibration")
cam0_dir = ds_dir / "cam0"
cam1_dir = ds_dir / "cam1"

cam0_ts = np.sort([int(x.name[0:-4]) for x in cam0_dir.glob("*.png")])
cam1_ts = np.sort([int(x.name[0:-4]) for x in cam1_dir.glob("*.png")])

# Output directory
out_dir = Path("datasets/calibration-out")
if out_dir.exists():
    shutil.rmtree(out_dir)

with tempfile.TemporaryDirectory() as tmp:
    tmp = Path(tmp)
    (tmp / "cam0").mkdir()
    (tmp / "cam1").mkdir()
    
    for t_l in tqdm(cam0_ts):
        t_r = cam1_ts[np.abs(cam1_ts - t_l).argmin()]
        if np.abs(t_l - t_r) < SYNC_TIME_DIFF_MAX * 1e6:
            # Convert to grayscale
            name = "%d.png" % t_l
            gray_l = cv2.cvtColor(cv2.imread(str(cam0_dir / name)), cv2.COLOR_BGR2GRAY)
            gray_r = cv2.cvtColor(cv2.imread(str(cam1_dir / ("%d.png" % t_r))), cv2.COLOR_BGR2GRAY)
            
            # Save
            cv2.imwrite(str(tmp / "cam0" / name), gray_l)
            cv2.imwrite(str(tmp / "cam1" / name), gray_r)
        else:
            print("Timestamp diff is too large:", (np.abs(t_l - t_r) / 1e9))
    
    shutil.copytree(tmp, out_dir)

  0%|          | 0/241 [00:00<?, ?it/s]

```
cd /path/to/calibration-dataset
docker pull ghcr.io/netdrones/kalibr:latest
docker run -it --rm -e DISPLAY \
    -v "/tmp/.X11-unix:/tmp/.X11-unix:ro" \
    -v `pwd`:/data ghcr.io/netdrones/kalibr
```

```
source devel/setup.bash
rosrun kalibr kalibr_bagcreater --folder /data --output-bag /data/cam.bag
rosrun kalibr kalibr_calibrate_cameras --bag /data/cam.bag --topics /cam0/image_raw /cam1/image_raw --models pinhole-radtan pinhole-radtan --target /data/aprilgrid_80x80cm.yaml 
```